In [1]:
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy as np
import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp
from pathlib import Path
import lossmaps as lm
import cernlayoutdb as layout
from cpymad.madx import Madx

 If you are on SWAN and did not expect this warning, then restart and load the 'NXCals Pro' software stack.


# Generating line from madx sequence WITH ERRORS - so non linearities

In [2]:
line = xt.Line.from_json('injection_lines/sps_lhc_q26_dipedge_rf.json')

Loading line from dict:   0%|          | 0/28198 [00:00<?, ?it/s]

Done loading line from dict.           


In [ ]:
line.get_table

AttributeError: 'Line' object has no attribute 'element_types'

In [3]:
line.to_madx_sequence(sequence_name='sps_dora')

KeyError: <class 'xtrack.beam_elements.apertures.LimitEllipse'>

In [2]:
OPTICSDIR = "/eos/user/l/lpauwels/acc-models-sps/"

p = 26  # beam momentum (GeV/c)

mad = Madx(stdout=True)

mad.call(file=f"{OPTICSDIR}/SPS_LS2_2020-05-26.seq")
mad.call(file=f"{OPTICSDIR}/strengths/lhc_q20.str") # change this depending on which optics you want
mad.call(file=f"{OPTICSDIR}/toolkit/macro.madx")

mad.command.beam(particle="PROTON", pc=p, charge=1)
mad.input("BRHO = BEAM->PC * 3.3356;")

mad.use(sequence='sps')
slice_kwargs = {'rbend':4, 'quadrupole':4, 'sbend':4} # here I am slicing every bend and quad into 4, change if you don't want so many slices
for class_, slice_ in slice_kwargs.items():
    mad.select(flag='makethin', class_=class_, slice_=slice_)
mad.makethin(sequence='sps', style='teapot', makedipedge=True)

mad.use(sequence="sps")
mad.input('exec, sps_set_tune_weights_q20;')
mad.input('exec, sps_set_chroma_weights_q20;')
mad.input('exec, sps_define_quad_knobs;')
mad.input('exec, sps_define_sext_knobs;')

mad.call(file=f"{OPTICSDIR}/sps_setMultipoles_upto7.cmd")
mad.input('exec, set_Multipoles_26GeV;')
mad.call(file=f"{OPTICSDIR}/sps_assignMultipoles_upto7.cmd")
mad.input('exec, AssignMultipoles;')

line = xt.Line.from_madx_sequence(mad.sequence["sps"], deferred_expressions=True, 
                                    apply_madx_errors=True, 
                                    install_apertures=False, 
                                    allow_thick=False)

line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV,
                                    gamma0=mad.sequence["sps"].beam.gamma)

line.vv['v200'] = 1e6 # total RF voltage
line.vv['freq200'] = 200e6 # RF frequency
line.vv['lag200'] = 180 # RF phase
line.element_refs['actcse.31632'].voltage = line.vars['v200']
line.element_refs['actcse.31632'].frequency = line.vars['freq200']
line.element_refs['actcse.31632'].lag = line.vars['lag200']


line.to_json("sps_lhc_q20_rf.json") # change the name of the line to whatever you want


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2025.02.19 15:02:59      +
  ++++++++++++++++++++++++++++++++++++++++++++




  /**********************************************************************************

  *

  * SPS Ring version (draft)LS2 in MAD X SEQUENCE format

  * Generated the 26-MAY-2020 14:48:33 from Layout

  *

  ***********************************************************************************/









/************************************************************************************/

/*                       TYPES DEFINITION                                           */

/************************************************************************************/



//---------------------- COLLIMATOR     ---------------------------------------------

BSHV      : COLLIMATOR  , L := .45;       ! beam scraper,

Converting sequence "sps":   0%|          | 0/14648 [00:00<?, ?it/s]

In [2]:
line = xt.Line.from_json("sps_lhc_q20_rf.json")

Loading line from dict:   0%|          | 0/14648 [00:00<?, ?it/s]

Done loading line from dict.           


In [4]:
len(line.element_names)

14648

In [4]:
print(line.element_names)

['sps$start', 'begi.10010', 'drift_0', 'qf.10010..1', 'drift_1', 'qf.10010..2', 'drift_2', 'qf.10010', 'drift_3', 'qf.10010..3', 'drift_4', 'qf.10010..4', 'drift_5', 'mba.10030_den', 'drift_6', 'mba.10030..1', 'drift_7', 'mba.10030..2', 'drift_8', 'mba.10030', 'drift_9', 'mba.10030..3', 'drift_10', 'mba.10030..4', 'drift_11', 'mba.10030_dex', 'drift_12', 'mba.10050_den', 'drift_13', 'mba.10050..1', 'drift_14', 'mba.10050..2', 'drift_15', 'mba.10050', 'drift_16', 'mba.10050..3', 'drift_17', 'mba.10050..4', 'drift_18', 'mba.10050_dex', 'drift_19', 'mbb.10070_den', 'drift_20', 'mbb.10070..1', 'drift_21', 'mbb.10070..2', 'drift_22', 'mbb.10070', 'drift_23', 'mbb.10070..3', 'drift_24', 'mbb.10070..4', 'drift_25', 'mbb.10070_dex', 'drift_26', 'mbb.10090_den', 'drift_27', 'mbb.10090..1', 'drift_28', 'mbb.10090..2', 'drift_29', 'mbb.10090', 'drift_30', 'mbb.10090..3', 'drift_31', 'mbb.10090..4', 'drift_32', 'mbb.10090_dex', 'drift_33', 'vvsa.10102', 'drift_34', 'lsd.10105', 'drift_35', 'mdv.10

In [ ]:
line.element_refs["mba.10030..1"]._get_value()

element_refs['mba.10030..1']

In [4]:
line["qf.10010..3"]

View of Multipole(_order=np.int64(1), inv_factorial_order=1, length=0.771, hxl=0, radiation_flag=np.int64(0), delta_taper=0, knl=array([0.        , 0.00893075]), ksl=array([0., 0.]), _sin_rot_s=-999, _cos_rot_s=-999, _shift_x=0, _shift_y=0, _shift_s=0, _internal_record_id=RecordIdentifier(buffer_id=np.int64(0), offset=np.int64(0)))

# Generating thick line with deferred expressions

In [4]:
OPTICSDIR = "/eos/user/l/lpauwels/acc-models-sps/"

p = 26  # beam momentum (GeV/c)

mad = Madx(stdout=True)

mad.call(file=f"{OPTICSDIR}/SPS_LS2_2020-05-26.seq")
mad.call(file=f"{OPTICSDIR}/strengths/lhc_q20.str") # change this depending on which optics you want
mad.call(file=f"{OPTICSDIR}/toolkit/macro.madx")

mad.command.beam(particle="PROTON", pc=p, charge=1)
mad.input("BRHO = BEAM->PC * 3.3356;")

mad.use(sequence="sps")
mad.input('exec, sps_set_tune_weights_q20;')
mad.input('exec, sps_set_chroma_weights_q20;')
mad.input('exec, sps_define_quad_knobs;')
mad.input('exec, sps_define_sext_knobs;')

line = xt.Line.from_madx_sequence(mad.sequence["sps"], deferred_expressions=True, 
                                    apply_madx_errors=False, 
                                    install_apertures=False, 
                                    allow_thick=True)

line.particle_ref = xt.Particles(mass0=xp.PROTON_MASS_EV,
                                    gamma0=mad.sequence["sps"].beam.gamma)

line.vv['v200'] = 1e6 # total RF voltage
line.vv['freq200'] = 200e6 # RF frequency
line.vv['lag200'] = 180 # RF phase
line.element_refs['actcse.31632'].voltage = line.vars['v200']
line.element_refs['actcse.31632'].frequency = line.vars['freq200']
line.element_refs['actcse.31632'].lag = line.vars['lag200']


line.to_json("injection_lines/injection_thick_no_ap_with_deffered_expressions.json") 


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.09.03  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2024.04.25               +
  + Execution date: 2025.02.24 11:48:35      +
  ++++++++++++++++++++++++++++++++++++++++++++




  /**********************************************************************************

  *

  * SPS Ring version (draft)LS2 in MAD X SEQUENCE format

  * Generated the 26-MAY-2020 14:48:33 from Layout

  *

  ***********************************************************************************/









/************************************************************************************/

/*                       TYPES DEFINITION                                           */

/************************************************************************************/



//---------------------- COLLIMATOR     ---------------------------------------------

BSHV      : COLLIMATOR  , L := .45;       ! beam scraper,

Converting sequence "sps":   0%|          | 0/3794 [00:00<?, ?it/s]


  Number of warnings: 0

  ++++++++++++++++++++++++++++++++++++++++++++
  +          MAD-X finished normally         +
  ++++++++++++++++++++++++++++++++++++++++++++
